<a href="https://colab.research.google.com/github/heroza/Keel/blob/main/yeast3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%load_ext autoreload
%autoreload 2

In [7]:
import myfunction as fu
import numpy as np
import pandas as pd
from typing_extensions import Counter
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn import svm
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split, StratifiedKFold
from imblearn.over_sampling import RandomOverSampler, SMOTE

In [8]:
df = pd.read_csv('yeast3.dat', header=None)


In [9]:
df.head()

,0,1,2,3,4,5,6,7,8
0,0.64,0.62,0.49,0.15,0.5,0.0,0.53,0.22,negative
1,0.58,0.44,0.57,0.13,0.5,0.0,0.54,0.22,negative
2,0.42,0.44,0.48,0.54,0.5,0.0,0.48,0.22,negative
3,0.51,0.40,0.56,0.17,0.5,0.5,0.49,0.22,negative
4,0.50,0.54,0.48,0.65,0.5,0.0,0.53,0.22,negative


In [10]:
num_features=8

le = LabelEncoder()
X = df.loc[:, df.columns[0:num_features]].to_numpy()

scaler = MinMaxScaler()
X = scaler.fit_transform(X)

y = df.loc[:, df.columns[-1]].to_numpy()
y = le.fit_transform(y)
# y_train = y_train.reshape(-1)
print(X.shape)
print(Counter(y))

(1484, 8)
Counter({0: 1321, 1: 163})


In [11]:
skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
thresholds = [0.02, 0.05, 0.1, 0.3, 0.5]
ks = [3,5,7,9]
skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

for k in ks:
  for threshold in thresholds:
    scores = []
    for i in range(3):
      for fold, (train_index, test_index) in enumerate(skf.split(X, y)):
        # print(f"Fold {fold+1}")
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        sia_model = fu.train_siamese_dense(X_train, y_train, X_test, y_test, num_classes=2, num_features=num_features, epochs_siamese=100, batch_size=16)

        # Initialize the SVM classifier
        clf = svm.SVC()

        X_train_resampled, y_train_resampled = fu.sia_smote_balance_dense(X_train, y_train, sia_model, threshold, maxdist_from_base=0.5, k = k)

        # Train the classifier on the training set
        clf.fit(X_train_resampled, y_train_resampled)

        # Predict the labels of the test set
        y_pred = clf.predict(X_test)

        # Calculate the F1 score
        score = fu.calculate_score(y_test, y_pred)
        scores.append(score)

        # Print the F1 score
        print(f"Score in i:{i}, fold:{fold}:", score)

    print(f'k={k} and threshold={threshold}')
    print('printing score')
    fu.print_score(scores)

creating pairs
Counter({1.0: 260, 0.0: 260})
Counter({1.0: 62, 0.0: 62})
init base network
training siamese network...
evaluate
4/4 [==============================] - 0s 3ms/step - loss: 0.1601
predict train
17/17 [==============================] - 0s 2ms/step
compute train acc
predict val
4/4 [==============================] - 0s 3ms/step
compute val acc
Loss = 0.1600930243730545, Train Accuracy = 0.9461538461538461 Test Accuracy = 0.8064516129032258
29/29 [==============================] - 0s 1ms/step
Score in i:0, fold:0: [0.9090909090909091, 0.9215801886792453, 0.9214426752509268, 0.6896551724137931, 0.5454545454545454, 0.9375, 0.9056603773584906, 0.9215801886792453]
creating pairs
Counter({1.0: 258, 0.0: 258})
Counter({1.0: 64, 0.0: 64})
init base network
training siamese network...
evaluate
4/4 [==============================] - 0s 3ms/step - loss: 0.1454
predict train
17/17 [==============================] - 0s 2ms/step
compute train acc
predict val
4/4 [========================

In [12]:
skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
thresholds = [0.02, 0.05, 0.1, 0.3, 0.5]
ks = [3,5,7,9]
skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

for k in ks:
  for threshold in thresholds:
    scores = []
    for i in range(3):
      for fold, (train_index, test_index) in enumerate(skf.split(X, y)):
        # print(f"Fold {fold+1}")
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        sia_model = fu.train_siamese_dense(X_train, y_train, X_test, y_test, num_classes=2, num_features=num_features, epochs_siamese=100, batch_size=16)

        # Initialize the SVM classifier
        clf = svm.SVC()

        X_train_resampled, y_train_resampled = fu.sia_smote_then_smote_dense(X_train, y_train, sia_model, threshold, maxdist_from_base=0.5, k = k)

        # Train the classifier on the training set
        clf.fit(X_train_resampled, y_train_resampled)

        # Predict the labels of the test set
        y_pred = clf.predict(X_test)

        # Calculate the F1 score
        score = fu.calculate_score(y_test, y_pred)
        scores.append(score)

        # Print the F1 score
        print(f"Score in i:{i}, fold:{fold}:", score)

    print(f'k={k} and threshold={threshold}')
    print('printing score')
    fu.print_score(scores)

creating pairs
Counter({1.0: 260, 0.0: 260})
Counter({1.0: 62, 0.0: 62})
init base network
training siamese network...
evaluate
4/4 [==============================] - 0s 3ms/step - loss: 0.1828
predict train
17/17 [==============================] - 0s 2ms/step
compute train acc
predict val
4/4 [==============================] - 0s 2ms/step
compute val acc
Loss = 0.182840034365654, Train Accuracy = 0.8884615384615384 Test Accuracy = 0.782258064516129
29/29 [==============================] - 0s 1ms/step
Score in i:0, fold:0: [0.9158249158249159, 0.9253537735849057, 0.925274054251873, 0.7058823529411765, 0.5660377358490566, 0.9375, 0.9132075471698113, 0.9253537735849057]
creating pairs
Counter({1.0: 258, 0.0: 258})
Counter({1.0: 64, 0.0: 64})
init base network
training siamese network...
evaluate
4/4 [==============================] - 0s 3ms/step - loss: 0.1214
predict train
17/17 [==============================] - 0s 1ms/step
compute train acc
predict val
4/4 [===========================